In [43]:
import netsquid as ns
from netsquid.nodes import Node, Network
from netsquid.components import QuantumChannel, ClassicalChannel, QuantumProcessor
from netsquid.qubits import qubitapi as qapi
from netsquid.qubits.qubitapi import create_qubits, operate, reduced_dm, measure
from sympy import Matrix
from sympy.printing.latex import latex
from IPython.display import display, Math

import socket
import pickle

In [44]:
# Function to print the density matrix of the qubits in a node's quantum memory
def print_density_matrix(node, qubit_indices):
    qubits = [node.qmemory.peek(i)[0] for i in qubit_indices]
    dm = reduced_dm(qubits)
    dm_matrix = Matrix(dm)
    print(f"Density matrix for {node.name}:")
    display(Math(latex(dm_matrix)))
    return dm

def print_density_matrix_whole_system(nodes):
    qubits = []
    for node in nodes:
        for i in range(node.qmemory.num_positions):
            qubit = node.qmemory.peek(i)[0]
            if qubit is not None:
                qubits.append(qubit)
    dm = reduced_dm(qubits)
    dm_matrix = Matrix(dm)
    print("Density matrix for the whole system:")
    display(Math(latex(dm_matrix)))
    return dm

# Function to create and distribute Bell states
def create_bell_state(node1, node2, pos1, pos2):
    qubits = create_qubits(2)
    qapi.operate(qubits[0], ns.H)
    qapi.operate([qubits[0], qubits[1]], ns.CX)
    node1.qmemory.put(qubits[0], positions=[pos1])
    node2.qmemory.put(qubits[1], positions=[pos2])

def bell_basis_measurement(node, pos1, pos2):
    qapi.operate([node.qmemory.peek(pos1)[0], node.qmemory.peek(pos2)[0]], ns.CX)
    qapi.operate(node.qmemory.peek(pos1)[0], ns.H)
    m1, _ = measure(node.qmemory.peek(pos1)[0])
    m2, _ = measure(node.qmemory.peek(pos2)[0])
    return m1, m2

In [45]:
# Create nodes for Alice, Charlie, and Bob with enough quantum memory slots
alice = Node("Alice", qmemory=QuantumProcessor("AliceProcessor", num_positions=1))
charlie = Node("Charlie", qmemory=QuantumProcessor("CharlieProcessor", num_positions=2))
bob = Node("Bob", qmemory=QuantumProcessor("BobProcessor", num_positions=1))

In [46]:
# Create a network and add the nodes
network = Network("Quantum Network")
network.add_nodes([alice, charlie, bob])

# Create quantum channels between the nodes
channel_ac = QuantumChannel("Alice_to_Charlie", length=10)
channel_ca = QuantumChannel("Charlie_to_Alice", length=10)
channel_cb = QuantumChannel("Charlie_to_Bob", length=10)
channel_bc = QuantumChannel("Bob_to_Charlie", length=10)

# Add quantum channels to the network
network.add_connection(alice, charlie, channel_to=channel_ac, channel_from=channel_ca, label="quantum")
network.add_connection(charlie, bob, channel_to=channel_cb, channel_from=channel_bc, label="quantum")

# Create classical channels for communication
classical_channel_ac = ClassicalChannel("Alice_to_Charlie_Classical", length=10)
classical_channel_ca = ClassicalChannel("Charlie_to_Alice_Classical", length=10)
classical_channel_cb = ClassicalChannel("Charlie_to_Bob_Classical", length=10)
classical_channel_bc = ClassicalChannel("Bob_to_Charlie_Classical", length=10)

# Add classical channels to the network
network.add_connection(alice, charlie, channel_to=classical_channel_ac, channel_from=classical_channel_ca, label="classical")
network.add_connection(charlie, bob, channel_to=classical_channel_cb, channel_from=classical_channel_bc, label="classical")

# Create and distribute Bell states
create_bell_state(alice, charlie, 0, 0)
create_bell_state(bob, charlie, 0, 1)

In [47]:
# Perform Bell basis measurement on Charlie's qubits
m1, m2 = bell_basis_measurement(charlie, 0, 1)
print(f"Bell basis measurement results for Charlie's qubits: {m1}, {m2}")

Bell basis measurement results for Charlie's qubits: 1, 0


In [48]:
alice_state = print_density_matrix(alice, [0])
bob_state = print_density_matrix(bob, [0])

#send alice and bob states by socket
packet = pickle.dumps([[alice_state, bob_state], [m1, m2]])

s = socket.socket()		 

port = 12346
host = 'localhost'		
s.bind((host, port))		 
s.listen(5)	 	 

# a forever loop until we interrupt it or 
# an error occurs 
while True: 

    # Establish connection with client. 
    c, addr = s.accept()	 
    print ('Got connection from', addr )

    c.send(packet) 

    # Close the connection with the client 
    c.close()

    # Breaking once connection closed
    break

Density matrix for Alice:


<IPython.core.display.Math object>

Density matrix for Bob:


<IPython.core.display.Math object>

Got connection from ('127.0.0.1', 52920)
